<a href="https://colab.research.google.com/github/vaibhavps05/Book-Recommendation-/blob/main/Book_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import warnings
warnings.simplefilter('ignore')

!pip install --upgrade pandas

import numpy as np
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 26.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [14]:
books = pd.read_csv('/content/BX-Books.csv', sep=";", warn_bad_lines=False, encoding='latin-1')
users = pd.read_csv('/content/BX-Users.csv', sep=";", warn_bad_lines=False, encoding='latin-1')
ratings = pd.read_csv('/content/BX-Book-Ratings.csv', sep=";", warn_bad_lines=False, encoding='latin-1')

TypeError: read_csv() got an unexpected keyword argument 'warn_bad_lines'

In [7]:
print("Books Dataset: ",books.shape)
print("Users Dataset: ",users.shape)
print("Ratings Dataset: ",ratings.shape)

NameError: name 'books' is not defined

In [ ]:
print("Books Dataset: ",books.columns)
print("\nUsers Dataset: ",users.columns)
print("\nRatings Dataset: ",ratings.columns)

In [ ]:
books.rename(columns={"Book-Title":'Title',
                      'Book-Author':'Author',
                     "Year-Of-Publication":'Year',
                     "Image-URL-L":"URL"},inplace=True)

ratings.rename(columns={"Book-Rating": "Ratings"}, inplace=True)

In [ ]:
books.head(2)

In [ ]:
users.head(2)

In [ ]:
ratings.head(2)

In [ ]:
books = books[['ISBN','Title', 'Author', 'Year', 'Publisher','URL']]

In [ ]:
books.head()

In [ ]:
ratings['User-ID'].value_counts()

In [ ]:
x = ratings['User-ID'].value_counts() > 200
x[x].shape

In [ ]:
y= x[x].index
ratings = ratings[ratings['User-ID'].isin(y)]
ratings.shape

In [ ]:
ratings_with_books = ratings.merge(books, on='ISBN')
ratings_with_books.head()

In [ ]:
ratings_with_books.shape

In [ ]:
number_rating = ratings_with_books.groupby('Title')['Ratings'].count().reset_index()
number_rating.head()

In [ ]:
number_rating.rename(columns={'Ratings':'No. of Ratings'},inplace=True)
number_rating.head()

In [ ]:
final_rating = ratings_with_books.merge(number_rating, on='Title')
final_rating.head()

In [ ]:
final_rating.shape


In [ ]:
final_rating = final_rating[final_rating['No. of Ratings'] >= 50]
final_rating.head()

In [ ]:
final_rating.shape

In [ ]:
final_rating.drop_duplicates(['User-ID','Title'],inplace=True)
final_rating.shape

In [ ]:
book_pivot = final_rating.pivot_table(columns='User-ID', index='Title', values= 'Ratings')
book_pivot

In [ ]:
book_pivot.shape

In [ ]:
book_pivot.fillna(0, inplace=True)
book_pivot

In [ ]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

book_names = book_pivot.index
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')
model.fit(book_sparse)

In [ ]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )
distance

In [ ]:
suggestion

In [ ]:
book_pivot.iloc[241,:]

In [ ]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

In [ ]:
import pickle
pickle.dump(model,open('Model.pkl','wb'))
pickle.dump(book_names,open('Book_Names.pkl','wb'))
pickle.dump(final_rating,open('Final_Rating.pkl','wb'))
pickle.dump(book_pivot,open('Book_Pivot.pkl','wb'))

In [ ]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )

    for i in range(len(suggestion)):
            books = book_pivot.index[suggestion[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)
book_name = "Harry Potter and the Chamber of Secrets (Book 2)"
recommend_book(book_name)